In [5]:
import pandas as pd
import os

# --- 1. CONFIGURAZIONE PERCORSI SICURA ---
# Usiamo il percorso assoluto della cartella corrente per non sbagliare
BASE_DIR = os.getcwd() # Cartella dove stai eseguendo lo script
OUTPUT_DIR = os.path.join(BASE_DIR, '../dataset')

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"📁 Cartella creata: {OUTPUT_DIR}")

# Cerchiamo il file di input
POSSIBILI_PERCORSI = [
    os.path.join(OUTPUT_DIR, 'VAERS_DEEP_CLEANED.csv'),
    os.path.join(BASE_DIR, 'VAERS_DEEP_CLEANED.csv'),
    'data/VAERS_DEEP_CLEANED.csv'
]

INPUT_FILE = None
for percorso in POSSIBILI_PERCORSI:
    if os.path.exists(percorso):
        INPUT_FILE = percorso
        print(f"✅ File input trovato in: {INPUT_FILE}")
        break

if INPUT_FILE is None:
    print("❌ ERRORE CRITICO: File input non trovato.")
    print(f"Ho cercato in: {POSSIBILI_PERCORSI}")
    raise FileNotFoundError("VAERS_DEEP_CLEANED.csv non trovato")

OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'VAERS_SANITY_CHECKED.csv')

print(f"Caricamento dati...")
try:
    df = pd.read_csv(INPUT_FILE, low_memory=False)
except Exception as e:
    print(f"❌ Errore nella lettura del file: {e}")
    raise

righe_iniziali = len(df)
print(f"Righe iniziali: {righe_iniziali}")

# --- 2. APPLICAZIONE REGOLE ---
print("\n--- INIZIO CONTROLLI DI COERENZA ---")

# ETÀ
mask_age = (df['AGE_YRS'] >= 0) & (df['AGE_YRS'] <= 110)
df = df[mask_age]
print(f"1. Età filtrata. Righe attuali: {len(df)}")

# GIORNI (Ho corretto il limite a 1000 come nel tuo codice)
if 'NUMDAYS' in df.columns:
    mask_days = (df['NUMDAYS'] >= 0) & (df['NUMDAYS'] <= 1000)
    df = df[mask_days]
    print(f"2. Giorni filtrati. Righe attuali: {len(df)}")

# SESSO
df = df[df['SEX'].isin(['M', 'F'])]
print(f"3. Sesso filtrato. Righe attuali: {len(df)}")

# DOSE
df = df[df['VAX_DOSE_SERIES'] >= 1]
print(f"4. Dose filtrata. Righe attuali: {len(df)}")

# TARGET
df = df[df['IS_SEVERE'].isin([0, 1])]
print(f"5. Target filtrato. Righe attuali: {len(df)}")

# --- 3. SALVATAGGIO ---
righe_finali = len(df)
perse = righe_iniziali - righe_finali

print("-" * 30)
print(f"SANITY CHECK COMPLETATO.")
print(f"Righe eliminate: {perse}")
print(f"Righe valide rimaste: {righe_finali}")

try:
    df.to_csv(OUTPUT_FILE, index=False)
    # Verifica immediata che il file esista
    if os.path.exists(OUTPUT_FILE):
        print(f"✅ SUCCESSO! File salvato in:\n   👉 {OUTPUT_FILE}")
        print(f"   Dimensione file: {os.path.getsize(OUTPUT_FILE) / 1024:.2f} KB")
    else:
        print("❌ ERRORE: Il comando ha finito ma il file non c'è. Controlla i permessi!")
except Exception as e:
    print(f"❌ ERRORE DURANTE IL SALVATAGGIO: {e}")
    print("Suggerimento: Chiudi il file se è aperto in Excel.")

✅ File input trovato in: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/VAERS_DEEP_CLEANED.csv
Caricamento dati...
Righe iniziali: 838912

--- INIZIO CONTROLLI DI COERENZA ---
1. Età filtrata. Righe attuali: 838904
2. Giorni filtrati. Righe attuali: 838083
3. Sesso filtrato. Righe attuali: 838083
4. Dose filtrata. Righe attuali: 838083
5. Target filtrato. Righe attuali: 838083
------------------------------
SANITY CHECK COMPLETATO.
Righe eliminate: 829
Righe valide rimaste: 838083
✅ SUCCESSO! File salvato in:
   👉 /Users/marcodonatiello/PycharmProjects/JupyterProject/data/VAERS_SANITY_CHECKED.csv
   Dimensione file: 154053.90 KB


In [9]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

# --- 1. CONFIGURAZIONE ---
BASE_DIR = os.getcwd()
# Input: Il Train Set che abbiamo creato nello step precedente
INPUT_FILE = os.path.join(BASE_DIR, '../dataset', 'train.csv')

OUTPUT_TRAIN_FINAL = os.path.join(BASE_DIR, '../dataset', 'train_final.csv')
OUTPUT_VAL = os.path.join(BASE_DIR, '../dataset', 'validation.csv')

print(f"Caricamento Train Set Originale: {INPUT_FILE}")
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError("ERRORE: Non trovo 'train.csv'. Hai fatto lo split precedente?")

df = pd.read_csv(INPUT_FILE, low_memory=False)

# --- 2. RICREAZIONE STRATIFICAZIONE (Bio-Logica) ---
# Dobbiamo ricreare la colonna STRATA perché nel file salvato non c'è più.
# Vogliamo che anche il Validation sia rappresentativo per età/sesso/gravità.

bins = [-1, 18, 64, 150]
labels = ['Young', 'Adult', 'Senior']
df['AGE_BIN'] = pd.cut(df['AGE_YRS'], bins=bins, labels=labels)

df['STRATA'] = (
    df['IS_SEVERE'].astype(str) + "_" +
    df['SEX'] + "_" +
    df['AGE_BIN'].astype(str)
)

# Gestione gruppi rari (come nello script precedente)
counts = df['STRATA'].value_counts()
rare_groups = counts[counts < 2].index
df.loc[df['STRATA'].isin(rare_groups), 'STRATA'] = df.loc[df['STRATA'].isin(rare_groups), 'IS_SEVERE'].astype(str)

# --- 3. ESECUZIONE SPLIT TRAIN / VALIDATION ---
# Solitamente si usa il 20% del Train come Validation (che equivale al 16% del totale originale)
VAL_SIZE = 0.20

print(f"\nGenerazione Validation Set ({VAL_SIZE*100}% del Train)...")

X = df.drop(columns=['IS_SEVERE'])
y = df['IS_SEVERE']

X_train_part, X_val, y_train_part, y_val = train_test_split(
    X, y,
    test_size=VAL_SIZE,
    random_state=42,
    stratify=df['STRATA'] # Stratificazione coerente
)

# --- 4. SALVATAGGIO ---
# Riuniamo i pezzi
train_final = X_train_part.copy()
train_final['IS_SEVERE'] = y_train_part

val_final = X_val.copy()
val_final['IS_SEVERE'] = y_val

# Pulizia colonne tecniche
cols_to_drop = ['STRATA', 'AGE_BIN']
train_final = train_final.drop(columns=cols_to_drop)
val_final = val_final.drop(columns=cols_to_drop)

train_final.to_csv(OUTPUT_TRAIN_FINAL, index=False)
val_final.to_csv(OUTPUT_VAL, index=False)

# --- 5. REPORT RIASSUNTIVO ---
print("-" * 40)
print(f"✅ SUDDIVISIONE COMPLETATA")
print("-" * 40)
print(f"1. Train Set FINALE (Studio):    {len(train_final)} righe")
print(f"   -> Salvato in: {OUTPUT_TRAIN_FINAL}")
print(f"2. Validation Set (Controllo):   {len(val_final)} righe")
print(f"   -> Salvato in: {OUTPUT_VAL}")
print("-" * 40)
print("📊 Percentuali di Gravità (Controllo Coerenza):")
perc_t = train_final['IS_SEVERE'].mean() * 100
perc_v = val_final['IS_SEVERE'].mean() * 100
print(f"   - % Gravi nel Train Final: {perc_t:.2f}%")
print(f"   - % Gravi nel Validation:  {perc_v:.2f}%")

Caricamento Train Set Originale: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/train.csv

Generazione Validation Set (20.0% del Train)...
----------------------------------------
✅ SUDDIVISIONE COMPLETATA
----------------------------------------
1. Train Set FINALE (Studio):    536370 righe
   -> Salvato in: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/train_final.csv
2. Validation Set (Controllo):   134093 righe
   -> Salvato in: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/validation.csv
----------------------------------------
📊 Percentuali di Gravità (Controllo Coerenza):
   - % Gravi nel Train Final: 12.84%
   - % Gravi nel Validation:  12.84%
